In [73]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import deepdish as dd
from pathlib import Path

# Load data

In [74]:
data_root = Path("J:/_Shared/teaching/tum_imaging_analysis/whole_brain")

traces = dd.io.load(str(data_root / "traces.h5"))

In [75]:
plt.figure()
plt.imshow(traces, cmap="gray_r", aspect="auto", vmin=0, vmax=0.5)

FigureCanvasNbAgg()

# Clustering

In [76]:
from sklearn.cluster import KMeans

In [77]:
from scipy.stats import zscore

In [78]:
n_clusters = 8
kmeans =  KMeans(n_clusters)

In [79]:
trs = np.nan_to_num(zscore(traces, 1))

In [80]:
clust_ids = kmeans.fit_predict(trs)

In [ ]:
plt.figure()
plt.plot(kmeans.cluster_centers_.T, lw=0.5)


In [ ]:
plt.figure()
plt.imshow(traces[np.argsort(clust_ids), :], cmap="gray_r", aspect="auto", vmin=0, vmax=0.5)

# Principal Components

# Display in anatomy

In [ ]:
from numba import jit

In [ ]:
anat = dd.io.load(str(data_root / "correlation_map.h5"))

In [ ]:
coords = dd.io.load(str(data_root / "coords.h5"))

In [ ]:
from notebook_utilities.display import display_array

In [ ]:
anat_rgb = np.empty(anat.shape+(3,), dtype=np.uint8)

In [ ]:
from utilities import normalise_clip, display_array

In [ ]:
anat_rgb[:, :, :, :] = 255-normalise_clip(anat, wins_min=10)[:, :, :, None]

In [ ]:
import math

In [ ]:

def paint_anatomy(stack, points, color=(230, 40, 0), **kwargs):
    color = np.array(color)
    return _paint_anatomy_jit(stack, points, color, **kwargs)


@jit(nopython=True)
def _paint_anatomy_jit(
    stack, points, color, kernel=(0.5, 1, 1), kernel_mult=2, minimal_weight=10,
):
    """ Extracts traces around points with a Gaussian profile

    :param stack: the stack from which the traces are to be extracted
    :param points: the points around which to extract the traces, shape
        [#points, 3]
    :param kernel: the standard deviations for the Gaussian used for weighting
    :param kernel_mult: how many standard deviations to take into account
    :return: an array with traces
    """
    for ip in range(points.shape[0]):
        point = points[ip]

        zs = min(
            max(int(math.floor(point[0] - kernel[0] * kernel_mult)), 0),
            stack.shape[0] - 1,
        )
        ze = min(
            max(int(math.ceil(point[0] + kernel[0] * kernel_mult)), 0),
            stack.shape[0] - 1,
        )
        ys = min(
            max(int(math.floor(point[1] - kernel[1] * kernel_mult)), 0),
            stack.shape[1] - 1,
        )
        ye = min(
            max(int(math.ceil(point[1] + kernel[1] * kernel_mult)), 0),
            stack.shape[1] - 1,
        )
        xs = min(
            max(int(math.floor(point[2] - kernel[2] * kernel_mult)), 0),
            stack.shape[2] - 1,
        )
        xe = min(
            max(int(math.ceil(point[2] + kernel[2] * kernel_mult)), 0),
            stack.shape[2] - 1,
        )

        # iterate throught the window in 3 dimensions
        for z in range(zs, ze + 1):
            for y in range(ys, ye + 1):
                for x in range(xs, xe + 1):

                    # calculate the weight from the Gaussian
                    w = (
                        math.exp(-(z - point[0]) ** 2 / (2 * kernel[0]))
                        * math.exp(-(y - point[1]) ** 2 / (2 * kernel[1]))
                        * math.exp(-(x - point[2]) ** 2 / (2 * kernel[2]))
                    )

                    stack[z, y, x, :] = stack[z, y, x, :] * (1-w) + color * w
    return stack

In [71]:
paint_anatomy(anat_rgb, coords, kernel=(0.8, 3, 3))

array([[[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [102, 102, 102],
         [ 66,  66,  66],
         ...,
         [ 74,  74,  74],
         [130, 130, 130],
         [255, 255, 255]],

        [[255, 255, 255],
         [ 82,  82,  82],
         [ 39,  39,  39],
         ...,
         [ 52,  52,  52],
         [120, 120, 120],
         [255, 255, 255]],

        ...,

        [[255, 255, 255],
         [ 84,  84,  84],
         [ 31,  31,  31],
         ...,
         [ 54,  54,  54],
         [112, 112, 112],
         [255, 255, 255]],

        [[255, 255, 255],
         [154, 154, 154],
         [140, 140, 140],
         ...,
         [163, 163, 163],
         [178, 178, 178],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
        

In [72]:
display_array(anat_rgb, vmin=0, invert=False)

interactive(children=(IntSlider(value=17, description='i', max=34), Output()), _dom_classes=('widget-interact'…

<function utilities.display_array.<locals>.browse(i: (0, 34))>